## Block Matching Regisrtation

This notebook outputs transformaed pointclouds + alignmnet matrices + warped images. 
This notebut is run in WSL since it is using ants , which is not available for windows. 

For vizualisation, go to " NC_Control_View" notebook , which is run from windows ... 

In [1]:
import sys
import pwreg as pw

import pickle
import pandas as pd

<frozen importlib._bootstrap>:219: RuntimeWarning: scipy._lib.messagestream.MessageStream size changed, may indicate binary incompatibility. Expected 56 from C header, got 64 from PyObject
/home/ply/anaconda3/envs/ants/lib/python3.8/site-packages/statsmodels/compat/pandas.py:65: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  from pandas import Int64Index as NumericIndex


In [2]:
project_path = "/mnt/d/Code/repos/pwreg"

### Global image alignment 
( TODO : allow rigid ) 

In [3]:
mask_tp2 = {'zmin': 51,'zmax': 164 + 1}
mask_tp1 = {'zmin': 51,'zmax': 168 + 1}

fixed = pw.Image([0.658, 0.23, 0.23], filename = f'{project_path}/data/raw/img/no_change_control/Image_1-20FJ.ome.tif', mask = mask_tp1) # tp 1
moving = pw.Image([0.658, 0.23, 0.23], filename = f'{project_path}/data/raw/img/no_change_control/Image_1-20FM.ome.tif', mask = mask_tp2) # tp 2

print(f'shape\nFixed {fixed.shape},\nMoving {moving.shape}')
print(f'masks :\nFixed {fixed.mask},\nMoving {moving.mask}')

shape
Fixed (118, 1024, 1024),
Moving (114, 1024, 1024)
masks :
Fixed {'zmin': 51, 'zmax': 169, 'xmin': None, 'xmax': None, 'ymin': None, 'ymax': None},
Moving {'zmin': 51, 'zmax': 165, 'xmin': None, 'xmax': None, 'ymin': None, 'ymax': None}


In [4]:
# put the whole image size here
voxels_f = fixed.split([118,1024,1024],0)
voxels_m = moving.split([114,1024,1024],0)

In [5]:
print(voxels_f)

[start [0 0 0]
size [ 118 1024 1024]
idx [0 0 0]
overlap [0 0 0]]


In [6]:
pair = pw.BlockPair(voxels_f[0],voxels_m[0])
pair.register(f"{project_path}/tmp",verbose = True) # block 2 (blc2) -to-> block 1 (blc1)

fixed : ANTsImage (RAI)
	 Pixel Type : float (float32)
	 Components : 1
	 Dimensions : (118, 1024, 1024)
	 Spacing    : (0.658, 0.23, 0.23)
	 Origin     : (0.0, 0.0, 0.0)
	 Direction  : [1.0 0.0 0.0 0.0 1.0 0.0 0.0 0.0 1.0]

moving : ANTsImage (RAI)
	 Pixel Type : float (float32)
	 Components : 1
	 Dimensions : (114, 1024, 1024)
	 Spacing    : (0.658, 0.23, 0.23)
	 Origin     : (0.0, 0.0, 0.0)
	 Direction  : [1.0 0.0 0.0 0.0 1.0 0.0 0.0 0.0 1.0]



In [7]:
tag = 'global_083022'
fixed.imwrite(f'{project_path}/data/processed/registered/no_change_control/testing_April/1-20FJ_{tag}.tif')
moving.imwrite(f'{project_path}/data/processed/registered/no_change_control/testing_April/1-20FM_{tag}.tif')
pair.warp().imwrite(f'{project_path}/data/processed/registered/no_change_control/testing_April/1-20FJ_to_1-20FM_{tag}.tif')


In [13]:

to_json(pair.alignment['affine'],f'{project_path}/data/processed/registered/no_change_control/testing_April/affinetransform_1-20FJ_to_1-20FM_{tag}.json')

### Global points alignment

In [16]:
segmentation_folder = '/mnt/d/Code/repos/psd95_segmentation/models/two_small_cnns/predictions/Gad1b_2021/Sequential_2tp_control/'

ptc1 = Points.from_predictions(f'{segmentation_folder}ML_segmentation_1-20GA_12-22-2021_2CNN_gui.csv', prob_thr = 0.6, units='pix', resolution=[0.658, 0.23, 0.23])
ptc2 = Points.from_predictions(f'{segmentation_folder}ML_segmentation_1-20G6_12-22-2021_2CNN_gui.csv', prob_thr = 0.0, units='pix', resolution=[0.658, 0.23, 0.23])

In [17]:
ptc1 = ptc1.crop(mask_tp1, units='pix')
ptc1 = ptc1.recenter([mask_tp1['zmin'],0,0], units = 'pix')

ptc2 = ptc2.crop(mask_tp2, units='pix')
ptc2 = ptc2.recenter([mask_tp2['zmin'],0,0], units = 'pix')

af = from_json(AffineTransform, f'{project_path}/data/processed/registered/no_change_control/testing_April/affinetransform_1-20FJ_to_1-20FM_{tag}.json')
ptc1_tf = ptc1.transform(af, units='phs')

to_json(ptc1, f'{project_path}/data/processed/registered/no_change_control/testing_April/ptc1_global_{tag}.json')
to_json(ptc2, f'{project_path}/data/processed/registered/no_change_control/testing_April/ptc2_global_{tag}.json')
to_json(ptc1_tf, f'{project_path}/data/processed/registered/no_change_control/testing_April/ptc1_to_ptc2_global_{tag}.json')

### Block affine registration

In [18]:
# use the pre-registered 
prereg = pair.warp(interpolator = 'linear')

Let's try with the overlap:

In [19]:
# you want to pick the overlap so that it is larger than the pairing radius that you will be using ... ( here I'm thinking of 1.5 um radius ) 
voxels_f = fixed.split([18,102,102],[3,7,7])
voxels_m = prereg.split([18,102,102],[3,7,7])

/mnt/d/Code/repos/pwreg/pwreg/core/core.py:108: UserWarning:

Specified voxel size + overlap don't cover the whole image.Image size is [ 118 1024 1024], block size [ 18 102 102], overlap [3 7 7] results in [ 7.66666667 10.70526316 10.70526316] blocks.
Leaving some image out. 



In [20]:
pairs = [BlockPair(voxf,voxm) for voxf,voxm in zip(voxels_f,voxels_m)]
for pair in pairs: 
    pair.register()

In [44]:
BlockPairView(pairs, 10).write_volume(f'{project_path}/data/processed/registered/no_change_control/pw_registration_moving_1-20FJ_overlap_NN_ZYX.tif')
BlockView(voxels_f,10).write_volume(f'{project_path}/data/processed/registered/no_change_control/pw_registration_fixed_1-20FM_overlap_NN_ZYX.tif')

### Block points registration

In [74]:
segmentation_folder = '/mnt/d/Code/repos/psd95_segmentation/models/two_small_cnns/predictions/Gad1b_2021/Sequential_2tp_control/'
ptc2 = Points.from_predictions(f'{segmentation_folder}ML_segmentation_1-20G6_12-22-2021_2CNN_gui.csv', prob_thr=0.5, units='pix', resolution=[0.658, 0.23, 0.23]) # tp2

ptc2 = ptc2.crop(mask_tp2, units='pix')
ptc2 = ptc2.recenter([51,0,0], units = 'pix')

In [82]:
ptc2_in_blocks = ptc2.split(voxels_f)
PointsBlockView(ptc2_in_blocks, voxels_f, 10).to_json(f'{project_path}/data/processed/registered/no_change_control/ptc2_block_ZYX.json')

In [90]:
ptc1_global = Points.from_json(f'{project_path}/data/processed/registered/no_change_control/ptc1_global_tansformed_to_ptc2_TS_resolution_ZYX.json')
ptc1_to_ptc2_in_blocks = ptc1_global.pw_transform(pairs)
PointsBlockView(ptc1_to_ptc2_in_blocks, voxels_f, 10).to_json(f'{project_path}/data/processed/registered/no_change_control/ptc1_to_ptc2_block_ZYX.json')